# Healthcare spending, part 2

In [1]:
#install.packages("lfe")
#install.packages("stargazer")
suppressWarnings(library(lfe))
suppressWarnings(library(stargazer))
suppressWarnings(library(dplyr))

Loading required package: Matrix

Please cite as: 

 Hlavac, Marek (2018). stargazer: Well-Formatted Regression and Summary Statistics Tables.
 R package version 5.2.2. https://CRAN.R-project.org/package=stargazer 


Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



## 1. import data and cleanup

In [2]:
getwd()
sample <- read.csv("data/health-spending.csv", nrow=10)
head(sample)
sapply(sample, class)
dim(sample)

[1] "C:/Users/wue04/Box Sync/healthcare-spending-fiscal-rules/part2-cost-shifting"

country,year,bailout,gdp,pop,debt,POP_80.,GOV_LEFT,FR,er_level,bbr_nat_2lvls,OOP_Exp,CURE,CARE,PREV,PHARM,BEDS,BEDS_LTC,CAP.FORMATION,HOSPITAL_EXP
Australia,1985,0,4.03e+11,15788310,NA,300.0,100,1,1,0,7255.928,14824.11,NA,254.089,1157.520,98534,NA,2432.782,9455.085
Australia,1986,0,4.11e+11,16018350,NA,314.3,100,1,1,0,8023.363,15381.10,NA,284.003,1296.217,NA,NA,2580.394,9785.289
Australia,1987,0,4.33e+11,16263870,NA,327.6,100,1,1,0,8325.094,15663.99,NA,320.267,1476.931,94653,NA,2718.716,10086.669
Australia,1988,0,4.53e+11,16532160,NA,340.5,100,1,1,0,9123.358,15856.88,NA,89.551,1462.585,NA,NA,2649.690,10011.853
Australia,1989,0,4.72e+11,16814420,80455919201,354.1,100,0,0,0,9763.080,16179.57,NA,118.617,1580.287,91312,NA,2663.733,9909.812
Australia,1990,0,4.78e+11,17065130,78639034361,367.3,100,0,0,0,10681.239,16821.24,NA,4.270,1511.820,NA,NA,2573.098,10036.322


country          year       bailout           gdp           pop 
     "factor"     "integer"     "integer"     "numeric"     "integer" 
         debt       POP_80.      GOV_LEFT            FR      er_level 
    "numeric"     "numeric"     "integer"     "integer"     "integer" 
bbr_nat_2lvls       OOP_Exp          CURE          CARE          PREV 
    "integer"     "numeric"     "numeric"     "logical"     "numeric" 
        PHARM          BEDS      BEDS_LTC CAP.FORMATION  HOSPITAL_EXP 
    "numeric"     "integer"     "logical"     "numeric"     "numeric"

[1] 10 20

In [4]:
health <- read.csv("data/health-spending.csv",
                  col.names=c("country", "year", "bailout", "gdp", "pop", "debt", "pop80",
                             "gov_left", "fr", "er", "bbr", "oop", "cure",
                             "care", "prev", "pharm", "beds", "beds_ltc", "cap", "hospital_exp", "rr"))
head(health)
health$country <- as.character(health$country)
sapply(health, class)

country,year,bailout,gdp,pop,debt,pop80,gov_left,fr,er,...,oop,cure,care,prev,pharm,beds,beds_ltc,cap,hospital_exp,rr
Australia,1985,0,4.03e+11,15788310,NA,300.0,100,1,1,...,7255.928,14824.11,NA,254.089,1157.520,98534,NA,2432.782,9455.085,1
Australia,1986,0,4.11e+11,16018350,NA,314.3,100,1,1,...,8023.363,15381.10,NA,284.003,1296.217,NA,NA,2580.394,9785.289,1
Australia,1987,0,4.33e+11,16263870,NA,327.6,100,1,1,...,8325.094,15663.99,NA,320.267,1476.931,94653,NA,2718.716,10086.669,1
Australia,1988,0,4.53e+11,16532160,NA,340.5,100,1,1,...,9123.358,15856.88,NA,89.551,1462.585,NA,NA,2649.690,10011.853,1
Australia,1989,0,4.72e+11,16814420,80455919201,354.1,100,0,0,...,9763.080,16179.57,NA,118.617,1580.287,91312,NA,2663.733,9909.812,0
Australia,1990,0,4.78e+11,17065130,78639034361,367.3,100,0,0,...,10681.239,16821.24,NA,4.270,1511.820,NA,NA,2573.098,10036.322,0


country         year      bailout          gdp          pop         debt 
 "character"    "integer"    "integer"    "numeric"    "numeric"    "numeric" 
       pop80     gov_left           fr           er          bbr          oop 
   "numeric"    "numeric"    "integer"    "integer"    "integer"    "numeric" 
        cure         care         prev        pharm         beds     beds_ltc 
   "numeric"    "numeric"    "numeric"    "numeric"    "numeric"    "numeric" 
         cap hospital_exp           rr 
   "numeric"    "numeric"    "integer"

In [5]:
# generate time lagged measuemrents
# t-1 and t-2
health <- health %>%
    group_by(country) %>%
    mutate(fr1 = lag(fr, 1))%>%
    mutate(fr2 = lag(fr, 2))%>%
    mutate(er1 = lag(er, 1))%>%
    mutate(er2 = lag(er, 2))%>%
    mutate(bbr1 = lag(bbr, 1))%>%
    mutate(bbr2 = lag(bbr, 2))%>%
    mutate(rr1 = lag(rr, 1))%>%
    mutate(rr2 = lag(rr, 2))

In [6]:
names(health)
head(health[, c("country", "bbr", "bbr1", "bbr2")])

[1] "country"      "year"         "bailout"      "gdp"          "pop"         
 [6] "debt"         "pop80"        "gov_left"     "fr"           "er"          
[11] "bbr"          "oop"          "cure"         "care"         "prev"        
[16] "pharm"        "beds"         "beds_ltc"     "cap"          "hospital_exp"
[21] "rr"           "fr1"          "fr2"          "er1"          "er2"         
[26] "bbr1"         "bbr2"         "rr1"          "rr2"

country,bbr,bbr1,bbr2
Australia,0,NA,NA
Australia,0,0,NA
Australia,0,0,0
Australia,0,0,0
Australia,0,0,0
Australia,0,0,0


In [10]:
# data characteristics
table(health$er)
table(health$bbr)


  0   1 
748 212 


  0   1 
689 271 

In [ ]:
rm(sample)

## 2. regressions - main models

In [38]:
# modify stargazer function to suppress output
mod_stargazer <- function(output.file, ...) {
  output <- capture.output(stargazer(...))
  cat(paste(output, collapse = "\n"), "\n", file=output.file, append=TRUE)
}

### budget shifting
##### log(out_of_pocket_exp) = ER + log(gdp) + log(debt) + log(pop) + bailout + fe(year) + fe(country)
### budget setting
##### log(cure) = ER + log(gdp) + log(debt) + log(pop) + bailout  + fe(year) + fe(country)
##### log(hospital_exp) = ER + log(gdp) + log(debt) + log(pop) + bailout  + fe(year) + fe(country)
##### log(care) = ER + log(gdp) + log(debt) + log(pop80) + bailout + fe(year) + fe(country)
##### log(prev) = ER + log(gdp) + log(debt) + log(pop) + bailout + gov_left + fe(year) + fe(country)
##### log(pharm) = ER + log(gdp) + log(debt) + log(pop) + bailout + fe(year) + fe(country)

In [45]:
shift_er <- felm(log(oop)~er+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health) #budget shifting
cure_er <- felm(log(cure)~er+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health) #budget setting
hosexp_er <- felm(log(hospital_exp)~er+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)
care_er <- felm(log(care)~er+log(debt)+log(gdp)+log(pop80)+bailout|as.factor(year)+as.factor(country),
             data=health)
prev_er <- felm(log(prev)~er+log(debt)+log(gdp)+log(pop)+bailout+gov_left|as.factor(year)+as.factor(country),
             data=health)
pharm_er <- felm(log(pharm)~er+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)
mod_stargazer(shift_er, cure_er, hosexp_er, care_er, prev_er, pharm_er,
              type="html",
              title="Table 1 - Budget Shifting and Budget Setting",
              dep.var.labels="Dependent variable:",
              dep.var.labels.include = FALSE,
              column.labels = c("OOP", "Curative", "Hospital expense", "Long term care",
                            "Preventative", "Pharmaceutical"),
              covariate.labels=c("ER", "log(Debt)", "log(GDP)", "log(Population)", "log(Pop 80+)",
                                 "IMF Bailout", "Left govt"),
              omit.stat = c("rsq", "ser"),
              notes = "All dependent variables are in natural log",
              out="output/rr/table1-budget.html")

### direct and indirect control of supply
##### log(beds) = FR/ER/BBR + log(gdp) + log(debt) + log(pop80) + bailout + fe(year) + fe(country)
##### log(capital) = FR/ER/BBR + log(gdp) + log(debt) + log(pop) + bailout + fe(year) + fe(country)

In [46]:
beds_er <- felm(log(beds)~er+log(debt)+log(gdp)+log(pop80)+bailout|as.factor(year)+as.factor(country),
             data=health)
cap_er <- felm(log(cap)~er+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)
mod_stargazer(beds_er, cap_er,
              type="html",
              title="Table 2 - Direct and Indirect Control of Supply",
              dep.var.labels="Dependent variable:",
              dep.var.labels.include = FALSE,
              column.labels = c("Hospital beds", "Capital"),
              covariate.labels=c("ER", "log(Debt)", "log(GDP)", "log(Pop 80+)", "log(Population)", 
                                 "IMF Bailout"),
              omit.stat = c("rsq", "ser"),
              notes = "All dependent variables are in natural log",
              out="output/rr/table2-supply_control.html")

### time lag, t-1 and t-2

In [48]:
shift_er1 <- felm(log(oop)~er1+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)
cure_er1 <- felm(log(cure)~er1+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)
hosexp_er1 <- felm(log(hospital_exp)~er1+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)
care_er1 <- felm(log(care)~er1+log(debt)+log(gdp)+log(pop80)+bailout|as.factor(year)+as.factor(country),
             data=health)
prev_er1 <- felm(log(prev)~er1+log(debt)+log(gdp)+log(pop)+bailout+gov_left|as.factor(year)+as.factor(country),
             data=health)
pharm_er1 <- felm(log(pharm)~er1+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)
beds_er1 <- felm(log(beds)~er1+log(debt)+log(gdp)+log(pop80)+bailout|as.factor(year)+as.factor(country),
             data=health)
cap_er1 <- felm(log(cap)~er1+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)
mod_stargazer(shift_er1, cure_er1, hosexp_er1, care_er1, prev_er1, pharm_er1, beds_er1, cap_er1,
              type="html",
              title="Table 3 - Time Lag One Year",
              dep.var.labels="Dependent variable:",
              dep.var.labels.include = FALSE,
              column.labels = c("OOP", "Curative", "Hospital expense", "Long term care",
                            "Preventative", "Pharmaceutical", "Beds", "Capital"),
              covariate.labels=c("ER", "log(Debt)", "log(GDP)", "log(Population)", "log(Pop 80+)",  
                                 "IMF Bailout", "Left govt"),
              omit.stat = c("rsq", "ser"),
              notes = "All dependent variables are in natural log",
              out="output/rr/table3-time_lag1.html")

In [49]:
shift_er2 <- felm(log(oop)~er2+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)
cure_er2 <- felm(log(cure)~er2+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)
hosexp_er2 <- felm(log(hospital_exp)~er2+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)
care_er2 <- felm(log(care)~er2+log(debt)+log(gdp)+log(pop80)+bailout|as.factor(year)+as.factor(country),
             data=health)
prev_er2 <- felm(log(prev)~er2+log(debt)+log(gdp)+log(pop)+bailout+gov_left|as.factor(year)+as.factor(country),
             data=health)
pharm_er2 <- felm(log(pharm)~er2+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)
beds_er2 <- felm(log(beds)~er2+log(debt)+log(gdp)+log(pop80)+bailout|as.factor(year)+as.factor(country),
             data=health)
cap_er2 <- felm(log(cap)~er2+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)
mod_stargazer(shift_er2, cure_er2, hosexp_er2, care_er2, prev_er2, pharm_er2, beds_er2, cap_er2, 
          type="html", 
          title="Table 4 - Time Lag Two Years",
          dep.var.labels="Dependent variable:",
          dep.var.labels.include = FALSE,
          column.labels = c("OOP", "Curative", "Hospital expense", "Long term care",
                            "Preventative", "Pharmaceutical", "Beds", "Capital"),
          covariate.labels=c("ER", "log(Debt)", "log(GDP)", "log(Population)", "log(Pop 80+)",
                             "IMF Bailout", "Left govt"),
          omit.stat = c("rsq", "ser"),
          notes = "All dependent variables are in natural log",
          out="output/rr/table4-time_lag2.html")

## 3. robustness check

#### using FR as main exposure

In [55]:
shift_fr <- felm(log(oop)~fr+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)
cure_fr <- felm(log(cure)~fr+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)
hosexp_fr <- felm(log(hospital_exp)~fr+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)
care_fr <- felm(log(care)~fr+log(debt)+log(gdp)+log(pop80)+bailout|as.factor(year)+as.factor(country),
             data=health)
prev_fr <- felm(log(prev)~fr+log(debt)+log(gdp)+log(pop)+bailout+gov_left|as.factor(year)+as.factor(country),
             data=health)
pharm_fr <- felm(log(pharm)~fr+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)
beds_fr <- felm(log(beds)~fr+log(debt)+log(gdp)+log(pop80)+bailout|as.factor(year)+as.factor(country),
             data=health)
cap_fr <- felm(log(cap)~fr+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)
mod_stargazer(shift_fr, cure_fr, hosexp_fr, care_fr, prev_fr, pharm_fr, beds_fr, cap_fr,
              type="html",
              title="Table 5 - Robustness Check, FR",
              dep.var.labels="Dependent variable:",
              dep.var.labels.include = FALSE,
              column.labels = c("OOP", "Curative", "Hospital expense", "Long term care",
                            "Preventative", "Pharmaceutical", "Beds", "Capital"),
              covariate.labels=c("ER", "log(Debt)", "log(GDP)", "log(Population)", "log(Pop 80+)",
                                 "IMF Bailout", "Left govt"),
              omit.stat = c("rsq", "ser"),
              notes = "All dependent variables are in natural log",
              out="output/rr/table5-robustness-fr.html")

#### using BBR as main exposure

In [54]:
shift_bbr <- felm(log(oop)~bbr+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)
cure_bbr <- felm(log(cure)~bbr+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)
hosexp_bbr <- felm(log(hospital_exp)~bbr+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)
care_bbr <- felm(log(care)~bbr+log(debt)+log(gdp)+log(pop80)+bailout|as.factor(year)+as.factor(country),
             data=health)
prev_bbr <- felm(log(prev)~bbr+log(debt)+log(gdp)+log(pop)+bailout+gov_left|as.factor(year)+as.factor(country),
             data=health)
pharm_bbr <- felm(log(pharm)~bbr+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)
beds_bbr <- felm(log(beds)~bbr+log(debt)+log(gdp)+log(pop80)+bailout|as.factor(year)+as.factor(country),
             data=health)
cap_bbr <- felm(log(cap)~bbr+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)
mod_stargazer(shift_bbr, cure_bbr, hosexp_bbr, care_bbr, prev_bbr, pharm_bbr, beds_bbr, cap_bbr,
              type="html",
              title="Table 6 - Robustness Check, BBR",
              dep.var.labels="Dependent variable:",
              dep.var.labels.include = FALSE,
              column.labels = c("OOP", "Curative", "Hospital expense", "Long term care",
                            "Preventative", "Pharmaceutical", "Beds", "Capital"),
              covariate.labels=c("ER", "log(Debt)", "log(GDP)", "log(Population)", "log(Pop 80+)",
                                 "IMF Bailout", "Left govt"),
              omit.stat = c("rsq", "ser"),
              notes = "All dependent variables are in natural log",
              out="output/rr/table6-robustness-bbr.html")

#### Time lagged measurements: FR t-1 and t-2

In [58]:
shift_fr1 <- felm(log(oop)~fr1+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)
cure_fr1 <- felm(log(cure)~fr1+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)
hosexp_fr1 <- felm(log(hospital_exp)~fr1+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)
care_fr1 <- felm(log(care)~fr1+log(debt)+log(gdp)+log(pop80)+bailout|as.factor(year)+as.factor(country),
             data=health)
prev_fr1 <- felm(log(prev)~fr1+log(debt)+log(gdp)+log(pop)+bailout+gov_left|as.factor(year)+as.factor(country),
             data=health)
pharm_fr1 <- felm(log(pharm)~fr1+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)
beds_fr1 <- felm(log(beds)~fr1+log(debt)+log(gdp)+log(pop80)+bailout|as.factor(year)+as.factor(country),
             data=health)
cap_fr1 <- felm(log(cap)~fr1+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)
mod_stargazer(shift_fr1, cure_fr1, hosexp_fr1, care_fr1, prev_fr1, pharm_fr1, beds_fr1, cap_fr1,
          type="html", 
          title="Table 7 - Robustness Check, FR Lag One Year",
          dep.var.labels="Dependent variable:",
          dep.var.labels.include = FALSE,
          column.labels = c("OOP", "Curative", "Hospital expense", "Long term care",
                            "Preventative", "Pharmaceutical", "Beds", "Capital"),
          covariate.labels=c("ER", "log(Debt)", "log(GDP)", "log(Population)", "log(Pop 80+)",
                             "IMF Bailout", "Left govt"),
          omit.stat = c("rsq", "ser"),
          notes = "All dependent variables are in natural log",
          out="output/rr/table7-robustness-fr1.html")

In [59]:
shift_fr2 <- felm(log(oop)~fr2+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)
cure_fr2 <- felm(log(cure)~fr2+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)
care_fr2 <- felm(log(care)~fr2+log(debt)+log(gdp)+log(pop80)+bailout|as.factor(year)+as.factor(country),
             data=health)
prev_fr2 <- felm(log(prev)~fr2+log(debt)+log(gdp)+log(pop)+bailout+gov_left|as.factor(year)+as.factor(country),
             data=health)
pharm_fr2 <- felm(log(pharm)~fr2+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)
beds_fr2 <- felm(log(beds)~fr2+log(debt)+log(gdp)+log(pop80)+bailout|as.factor(year)+as.factor(country),
             data=health)
cap_fr2 <- felm(log(cap)~fr2+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)
hosexp_fr2 <- felm(log(hospital_exp)~fr2+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)
mod_stargazer(shift_fr2, cure_fr2, hosexp_fr2, care_fr2, prev_fr2, pharm_fr2, beds_fr2, cap_fr2,
          type="html", 
          title="Table 8 - Robustness Check, FR Lag Two Years",
          dep.var.labels="Dependent variable:",
          dep.var.labels.include = FALSE,
          column.labels = c("OOP", "Curative", "Hospital expense", "Long term care",
                            "Preventative", "Pharmaceutical", "Beds", "Capital"),
          covariate.labels=c("ER", "log(Debt)", "log(GDP)", "log(Population)", "log(Pop 80+)",
                             "IMF Bailout", "Left govt"),
          omit.stat = c("rsq", "ser"),
          notes = "All dependent variables are in natural log",
          out="output/rr/table8-robustness-fr2.html")

In [62]:
ls()

[1] "health"

In [61]:
rm(list=ls(pattern="\\_"))

## 3. R&R

### run radomization simulatioin

### summary statistics